In [9]:
import pandas as pd
import numpy as np
import string
import re
from unidecode import unidecode

# Our sklearn imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

#Text
import emoji
from emoji.unicode_codes import UNICODE_EMOJI


# pip install nltk
# nltk.download()
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer, SnowballStemmer
import time

In [10]:
stops = stopwords.words('english')
vec = CountVectorizer()

In [11]:
df=pd.read_json("data_final.json", lines=True)
#tweet=df['full_text']

In [12]:
list(df.columns) 

['created_at',
 'id',
 'id_str',
 'full_text',
 'truncated',
 'display_text_range',
 'entities',
 'metadata',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'lang',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status',
 'possibly_sensitive',
 'extended_entities']

In [24]:
pd.set_option('display.max_colwidth',1000)
df[["metadata","coordinates","place"]].head()

,metadata,coordinates,place
0,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,"{'id': '5c6bd208d7ddf9f6', 'url': 'https://api.twitter.com/1.1/geo/id/5c6bd208d7ddf9f6.json', 'place_type': 'city', 'name': 'Cary', 'full_name': 'Cary, NC', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-78.9448945, 35.6443837], [-78.7291467, 35.6443837], [-78.7291467, 35.867993], [-78.9448945, 35.867993]]]}, 'attributes': {}}"
1,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,"{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api.twitter.com/1.1/geo/id/fbd6d2f5a4e4a15e.json', 'place_type': 'admin', 'name': 'California', 'full_name': 'California, USA', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-124.482003, 32.528832], [-114.131212, 32.528832], [-114.131212, 42.009519], [-124.482003, 42.009519]]]}, 'attributes': {}}"
2,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,"{'id': 'cb74aaf709812e0f', 'url': 'https://api.twitter.com/1.1/geo/id/cb74aaf709812e0f.json', 'place_type': 'city', 'name': 'Tulsa', 'full_name': 'Tulsa, OK', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-96.0656278, 35.968624], [-95.761656, 35.968624], [-95.761656, 36.250159], [-96.0656278, 36.250159]]]}, 'attributes': {}}"
3,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,"{'id': '01d62f216bc2c949', 'url': 'https://api.twitter.com/1.1/geo/id/01d62f216bc2c949.json', 'place_type': 'city', 'name': 'Stillwater', 'full_name': 'Stillwater, NY', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-73.698693, 42.9091943], [-73.630371, 42.9091943], [-73.630371, 42.959595], [-73.698693, 42.959595]]]}, 'attributes': {}}"
4,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,"{'id': 'a592bd6ceb1319f7', 'url': 'https://api.twitter.com/1.1/geo/id/a592bd6ceb1319f7.json', 'place_type': 'city', 'name': 'San Diego', 'full_name': 'San Diego, CA', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-117.282538, 32.53962], [-116.9274403, 32.53962], [-116.9274403, 33.0804044], [-117.282538, 33.0804044]]]}, 'attributes': {}}"


In [23]:
df.place.head()

0                {'id': '5c6bd208d7ddf9f6', 'url': 'https://api.twitter.com/1.1/geo/id/5c6bd208d7ddf9f6.json', 'place_type': 'city', 'name': 'Cary', 'full_name': 'Cary, NC', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-78.9448945, 35.6443837], [-78.7291467, 35.6443837], [-78.7291467, 35.867993], [-78.9448945, 35.867993]]]}, 'attributes': {}}
1    {'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api.twitter.com/1.1/geo/id/fbd6d2f5a4e4a15e.json', 'place_type': 'admin', 'name': 'California', 'full_name': 'California, USA', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-124.482003, 32.528832], [-114.131212, 32.528832], [-114.131212, 42.009519], [-124.482003, 42.009519]]]}, 'attributes': {}}
2                  {'id': 'cb74aaf709812e0f', 'url': 'https://api.twitter.com/1.1/geo/id/cb74aaf709812e0f.json', 'place_type': 'city', 'name': '

In [213]:
#df[0:1]

Word Processing (Stem, stopwords,...)

In [25]:
stemmer = PorterStemmer()

# Preprocessing
def preprocess(text):
    
    text = unidecode(text)
    
    # Lowercase
    text = text.lower()
    
    #emoji to text
    text = emoji.demojize(text)
    
    
    # Remove punctuation.
    punc = string.punctuation
    #.replace("@", "").replace("#", "")
    ##@ and hashtag are meaningful in tweets
    text = text.translate(punc)
    text = re.sub("\"|\.|\?|\!|\,|\-|\_|\(|\)|\{|\}|\/|\=|\+|\:", " ", text)
    text = text.replace("<br />", " ")


    # Strip whitespace
    text = [x.strip() for x in text.replace("\n", " ").split(" ")]
    text = [x for x in text if x]
    
    # Remove stopwords
    text = [x for x in text if x not in stops]
    
    # Stemming
    text = [stemmer.stem(x) for x in text]
    
    # Election
    
#     trump=0
#     biden=0
#     president=0
#     election=0
    
#     for x in text:
#         if x.find('trump')>0:
#             trump+=1
#         if x.find('biden')>0:
#             biden+=1
#         if x.find('president')>0:
#             president+=1
#         if x.find('election')>0:
#             election+=1
        
# Results
    text = " ".join(text)
    
#     if trump>0:
#         text = text+' trump'
#     if biden>0:
#         text = text+' biden'
#     if president>0:
#         text = text+' president'
#     if election>0:
#         text = text+' election'
    
    text = text.strip()
    
    return text

In [26]:
df["clean"] = df.full_text.map(preprocess)

In [29]:
df.full_text[3]

"You really can't make this stuff up. As Columbia County suffers a #COVID19 spike due to golfing events. Sen. Jordan shows us just what those events looked like by attending a political golf event in Easton with NO MASKS and NO SOCIAL DISTANCING. #NewYork #MaskUp https://t.co/zeOe511KdM"

In [30]:
df.clean[3]

"realli can't make stuff columbia counti suffer #covid19 spike due golf event sen jordan show us event look like attend polit golf event easton mask social distanc #newyork #maskup http co zeoe511kdm"

Vectorize words

In [31]:
#Vectors of variables 
vec = CountVectorizer()
dtm = vec.fit_transform(df.clean)
featnames = vec.get_feature_names()
dtm

<40000x70316 sparse matrix of type '<class 'numpy.int64'>'
	with 670801 stored elements in Compressed Sparse Row format>

In [33]:
#print(dtm[1:10,:])
results_df = pd.DataFrame(dtm.toarray(), columns=vec.get_feature_names())
featsums = results_df.sum()
# results_df.head()

In [34]:
#featsums[0:5]

00                    52
000                 1000
0000                   1
0000000000000001       1
0000001                1
dtype: int64

In [36]:
sum(featsums>100)

1093

In [149]:
#word_freq_max = results_df.max(axis=1)

In [38]:
#word_freq_max.mean()
featnames

['00',
 '000',
 '0000',
 '0000000000000001',
 '0000001',
 '000001',
 '0000111',
 '00019',
 '0003',
 '000322591645354',
 '00037',
 '00052',
 '00062',
 '00079',
 '0009',
 '000americansdead',
 '000death',
 '000iu',
 '000k',
 '000sq',
 '000th',
 '000x',
 '001',
 '00156',
 '002',
 '003',
 '004',
 '005',
 '007dossier',
 '008',
 '009',
 '00auus6400',
 '00bzse1cfp',
 '00cutoff',
 '00f5lbv6i9',
 '00ga21zdsz',
 '00ihsazr75',
 '00kd4enqtm',
 '00o',
 '00pm',
 '00pn00',
 '00y4oe8iem',
 '01',
 '01126',
 '012',
 '014',
 '015',
 '0172',
 '0174',
 '018',
 '01cybvegaz',
 '01urmievgp',
 '02',
 '020',
 '022',
 '023',
 '024',
 '0270',
 '028',
 '029',
 '02a5dzynng',
 '02g9kpxjz0',
 '02llhyqooo',
 '02nd',
 '02qcctmzcb',
 '02r5m0lonu',
 '02ztnz8lvq',
 '03',
 '030',
 '032',
 '035',
 '036',
 '038',
 '0380',
 '039',
 '03dpbea3bo',
 '03f3j4nqvg',
 '03pm',
 '03vvbqtsd5',
 '04',
 '040',
 '0417tina',
 '0444el',
 '045',
 '047',
 '04cliziapo',
 '04fauyusuw',
 '04kleq5wqc',
 '04tsbfv6iw',
 '04umnxp4i',
 '04z1pzzkj2',
 

In [40]:
#Top Freq Variables
select_words = 500
ordered_feats = featsums.argsort()[::-1]
feat_name=[]
print_list2=[]
for i in ordered_feats[:select_words]:
    feat_name.append(featnames[i])
    print_list2.append(featsums[i])
    print("Feat: {}, {}".format(featnames[i], featsums[i]))

Feat: covid, 20429
Feat: co, 20079
Feat: http, 19980
Feat: pandem, 10952
Feat: 19, 6859
Feat: covid19, 4587
Feat: get, 4534
Feat: peopl, 4134
Feat: amp, 4090
Feat: go, 3613
Feat: coronaviru, 3599
Feat: like, 3254
Feat: trump, 3094
Feat: test, 3069
Feat: one, 2595
Feat: time, 2486
Feat: mask, 2413
Feat: day, 2290
Feat: us, 2209
Feat: need, 2202
Feat: work, 2176
Feat: school, 2075
Feat: new, 2044
Feat: make, 1987
Feat: know, 1965
Feat: death, 1953
Feat: year, 1944
Feat: would, 1944
Feat: say, 1927
Feat: back, 1820
Feat: take, 1774
Feat: want, 1774
Feat: case, 1759
Feat: corona, 1757
Feat: think, 1669
Feat: still, 1637
Feat: die, 1629
Feat: state, 1573
Feat: realdonaldtrump, 1512
Feat: live, 1495
Feat: thing, 1462
Feat: american, 1451
Feat: help, 1437
Feat: see, 1402
Feat: even, 1401
Feat: can, 1368
Feat: got, 1358
Feat: viru, 1346
Feat: month, 1340
Feat: right, 1332
Feat: today, 1313
Feat: due, 1297
Feat: come, 1292
Feat: vote, 1290
Feat: thank, 1274
Feat: week, 1261
Feat: mani, 1207
Fea

In [37]:
#print_feat=pd.DataFrame([print_list1,print_list2])
#print_feat.to_excel('top200.xlsx', header=False, index=False)

In [41]:
select_list=results_df[feat_name]

In [42]:
select_list.shape

(40000, 500)

Correlation

In [43]:
corrMatrix = select_list.corr()

In [47]:
c = corrMatrix
s = c.unstack()
so = s.sort_values(kind="quicksort",ascending=False)


In [53]:
type(so)

pandas.core.series.Series

In [56]:
pd.set_option('display.max_rows', None)

In [57]:
so[(so<1)& (so>0.2)]

http         co             0.995548
co           http           0.995548
wear         mask           0.603389
mask         wear           0.603389
social       distanc        0.587366
distanc      social         0.587366
kamalaharri  joebiden       0.466837
joebiden     kamalaharri    0.466837
term         long           0.436433
long         term           0.436433
19           covid          0.430552
covid        19             0.430552
mail         vote           0.408358
vote         mail           0.408358
zealand      100            0.403103
100          zealand        0.403103
footbal      colleg         0.386879
colleg       footbal        0.386879
york         new            0.375742
new          york           0.375742
test         posit          0.368002
posit        test           0.368002
execut       order          0.350390
order        execut         0.350390
here         zealand        0.348715
zealand      here           0.348715
new          zealand        0.345782
z

In [58]:
so[(so<(-.2))]

19          pandem       -0.206041
pandem      19           -0.206041
coronaviru  covid        -0.237092
covid       coronaviru   -0.237092
            covid19      -0.278138
covid19     covid        -0.278138
pandem      covid        -0.485349
covid       pandem       -0.485349
dtype: float64

In [59]:
to_delete=pd.DataFrame(so[(so<(-.2))]).reset_index()

In [60]:
to_delete['level_0'].unique().tolist()

['19', 'pandem', 'coronaviru', 'covid', 'covid19']

In [61]:
del select_list['19']
del select_list['pandem']
del select_list['covid']
del select_list['coronaviru']
del select_list['covid19']
del select_list['http']
del select_list['co']
del select_list['corona']
del select_list['viru']

In [62]:
select_list.shape

(40000, 491)

Topic Modeling

In [63]:
sel_name=list(select_list.columns)

In [65]:
lda_input=select_list.to_numpy()

In [66]:
lda_input.shape

(40000, 491)

In [71]:
start_time = time.time()

# Define Search Param
search_params = {'n_components': [2, 5, 8], 'learning_decay': [.7]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=10, learning_method='online', 
                                learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(lda_input)

time.time()-start_time

729.1291399002075

In [73]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(lda_input))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 2}
Best Log Likelihood Score:  -367419.5780681907
Model Perplexity:  377.4749984602671


In [360]:
#lda_output

In [80]:
start_time = time.time()
lda_model = LatentDirichletAllocation(n_components=3,             # Number of topics
                                      max_iter=10,                # Max learning iterations
                                      learning_method='online',   # For large set
                                      #random_state=100,          # Random state
                                      batch_size=300,    # n docs in each learning iter
                                      n_jobs = -1,  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(lda_input)
time.time()-start_time
#print(lda_model)

37.47123193740845

In [81]:
# Display the top groups of words
def display_topics(model, feature_names, no_top_words):
    topic_collector = {}
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        topic_collector[topic_idx] = " ".join([feature_names[i]
                        for i in topic.argsort()[:-5 - 1:-1]])
    return topic_collector

no_top_words = 30
_ = display_topics(lda_model, sel_name, no_top_words)

Topic 0:
get like go peopl mask know think would thing make want can wear right play one we realli even sinc footbal time well see fuck that look hope way never
Topic 1:
test amp school work day still time get month thank today week home posit kid one health year back last help famili vaccin good student got need us feel safe
Topic 2:
trump new death case realdonaldtrump state vote american amp peopl die us 000 respons via number plan elect presid report kill need say america mail news million countri lie unemploy


In [82]:
# column names
topicnames = ['Topic' + str(i) for i in range(lda_model.n_components)]
# index names
docnames = ['Tweet_' + str(i) for i in range(len(lda_input))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [83]:
df_document_topic.head()

,Topic0,Topic1,Topic2,dominant_topic
Tweet_0,0.28,0.69,0.03,1
Tweet_1,0.20,0.38,0.43,2
Tweet_2,0.62,0.34,0.04,0
Tweet_3,0.54,0.02,0.44,0
Tweet_4,0.59,0.04,0.36,0


In [84]:
df_document_topic.groupby(['dominant_topic']).size()

dominant_topic
0    16738
1    13186
2    10076
dtype: int64

In [91]:
lda_input[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,